In [1]:
!pip install mediapipe 

In [2]:
import mediapipe as mp
import cv2

import csv
import os
import numpy as np
import pickle
import pandas as pd

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [26]:
landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value]

x: 0.7411400675773621
y: 0.8865759372711182
z: -0.2518969178199768
visibility: 0.9992336630821228

In [27]:
landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value]

x: 0.8360599279403687
y: 1.32499361038208
z: -0.1836368292570114
visibility: 0.13034459948539734

In [43]:
landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value]

x: 0.8302785158157349
y: 1.720853328704834
z: -0.5017535090446472
visibility: 0.036943137645721436

In [40]:
a = np.array([landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y])
b = np.array([landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y])
c = np.array([landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].y])

In [46]:
radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
radians

3.3694110613142607

In [44]:
radians_test = np.arctan2(0.4,-0.0058)
radians_test

1.5852953107147387

In [47]:
radians * 180 / np.pi

193.05303325800259

In [18]:
elbow = [landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y]

#Position of displaying the angle
angle_position_list = list(np.multiply(elbow, [1280, 720]).astype(int))
angle_position_list[0] = angle_position_list[0]+10
angle_position_tuple = tuple(angle_position_list)
print(angle_position_tuple)

(999, 889)


In [11]:
from platform import python_version

print(python_version())

3.8.8


# Capture left & right angles from a webcam

In [5]:
cap = cv2.VideoCapture(0)

#Points counter variables
position_Left = None
position_Right = None
counter = 0
frameCounter = 0
time = 0 
responseTime = 0

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    
    #this depends on the output video that is going to be put | the video and the internal camera need to be with the same size
    cap.set(3, 1280)
    cap.set(4, 720)
    
    print(cap.get(3))
    print(cap.get(4))
    print(cap.get(cv2.CAP_PROP_FPS))
        
    while cap.isOpened():
        ret, frame = cap.read()
        

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #ROI get
        #image1 = image[307:478, 591:714]
        
        #Adjust the results and the mediapipe to only watch at a given ROI (not in the whole image/video)
        results = holistic.process(image)
        #print(results.pose_landmarks)
            
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        #Extract the coordinates and calculating the angle
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for Left Hand
            hip = [landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].y]
            shoulder = [landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y]
          
            # Get coordinates for Right Hand
            hip_right = [landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].y]
            shoulder_right = [landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].y]
            
            # Calculate angles
            angle = calculate_angle(hip, shoulder, elbow)
            angle_right = calculate_angle(hip_right, shoulder_right, elbow_right)
            
            #Position of displaying the angle for the Left Side
            angle_position_list = list(np.multiply(shoulder, [1280, 720]).astype(int))
            angle_position_list[0] = angle_position_list[0]+30 #Placing it 30px on the right
            angle_position_tuple = tuple(angle_position_list)
            
            #Position of displaying the angle for the Right Side
            angle_right_position_list = list(np.multiply(shoulder_right, [1280, 720]).astype(int))
            angle_right_position_list[0] = angle_right_position_list[0]-30 #Placing it 30px on the left
            angle_right_position_tuple = tuple(angle_right_position_list)
            
            # Visualize angle
            cv2.putText(image, str(angle.astype(int)) + " C", 
                           angle_position_tuple, 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(image, str(angle_right.astype(int)) + " C", 
                           angle_right_position_tuple, 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            #Frame Counter
            frameCounter += 1
            
            #Time calculated based on the fps
            time = frameCounter / 15
            time = round(time, 2)
            
            #Check for the response Time 
    
            
            
            # Calculating the position and  the points for the Left Hand
            if angle < 25:
                position_Left = "NoN"
                
            elif angle >= 25 and angle < 60:
                position_Left = "1_P Left"

            elif angle >= 60 and angle < 110:
                position_Left = "2_P Left"

            elif angle >= 110:
                position_Left = "3_P Left"
                
            # Calculating the position and  the points for the Left Hand
            if angle_right < 25:
                position_Right = "NoN"
                
            elif angle_right >= 25 and angle_right < 60:
                position_Right = "1_P Right"

            elif angle_right >= 60 and angle_right < 110:
                position_Right = "2_P Right"

            elif angle_right >= 110:
                position_Right = "3_P Right"

                
            #Things that will NOT work
            #     To wait for the NoN position and then to take the last position and set the points (from position 3 to NoN
            #         it needs to move to 2 and then 1- so the last will be 1 position)
            #     To wait a bit in every position and if the position is held more than 1 sec than update the score
            
        except:
            pass
        
        #Render some text in the screen
        cv2.putText(image, str(counter), (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.putText(image, position_Right, (10,120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
                
        #On the middle of the screen 
        cv2.putText(image, str(time), (600,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
        #On the right of the screen
        cv2.putText(image, "Frames->" + str(frameCounter), (900,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, position_Left, (900,120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
                    
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()

640.0
480.0
1280.0
720.0
30.0


In [15]:
cap.release()
cv2.destroyAllWindows()

# Capture angles from a Video

In [6]:
videoKarate = "France_Spain_2010_Referee_Only.mp4"

In [8]:
cap = cv2.VideoCapture(videoKarate)

#Points counter variables
position_Left = None
position_Right = None
counter = 0
frameCounter = 0
time = 0 
responseTime = 0

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    
    #this depends on the output video that is going to be put | the video and the internal camera need to be with the same size
    cap.set(3, 1280)
    cap.set(4, 720)
    
    print(cap.get(3))
    print(cap.get(4))
    print("FPS -> " + str(cap.get(cv2.CAP_PROP_FPS)))
        
    while cap.isOpened():
        ret, frame = cap.read()
        

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #ROI get
        #image1 = image[55:561, 438:925]
        
        #Adjust the results and the mediapipe to only watch at a given ROI (not in the whole image/video)
        results = holistic.process(image)
        #print(results.pose_landmarks)
            
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        #Extract the coordinates and calculating the angle
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for Left Hand
            hip = [landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].y]
            shoulder = [landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y]
          
            # Get coordinates for Right Hand
            hip_right = [landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].y]
            shoulder_right = [landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].y]
            
            # Calculate angles
            angle = calculate_angle(hip, shoulder, elbow)
            angle_right = calculate_angle(hip_right, shoulder_right, elbow_right)
            
            #Position of displaying the angle for the Left Side
            angle_position_list = list(np.multiply(shoulder, [1280, 720]).astype(int))
            angle_position_list[0] = angle_position_list[0]+30 #Placing it 30px on the right
            angle_position_tuple = tuple(angle_position_list)
            
            #Position of displaying the angle for the Right Side
            angle_right_position_list = list(np.multiply(shoulder_right, [1280, 720]).astype(int))
            angle_right_position_list[0] = angle_right_position_list[0]-30 #Placing it 30px on the left
            angle_right_position_tuple = tuple(angle_right_position_list)
            
            # Visualize angle
            cv2.putText(image, str(angle.astype(int)) + " C", 
                           angle_position_tuple, 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(image, str(angle_right.astype(int)) + " C", 
                           angle_right_position_tuple, 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            #Frame Counter
            frameCounter += 1
            
            #Time calculated based on the fps
            time = frameCounter / 15
            time = round(time, 2)
            
            #Check for the response Time 
    
            
            
            # Calculating the position and  the points for the Left Hand
            if angle < 25:
                position_Left = "NoN"
                
            elif angle >= 25 and angle < 60:
                position_Left = "1_P Left"

            elif angle >= 60 and angle < 110:
                position_Left = "2_P Left"

            elif angle >= 110:
                position_Left = "3_P Left"
                
            # Calculating the position and  the points for the Left Hand
            if angle_right < 25:
                position_Right = "NoN"
                
            elif angle_right >= 25 and angle_right < 60:
                position_Right = "1_P Right"

            elif angle_right >= 60 and angle_right < 110:
                position_Right = "2_P Right"

            elif angle_right >= 110:
                position_Right = "3_P Right"

                
            #Things that will NOT work
            #     To wait for the NoN position and then to take the last position and set the points (from position 3 to NoN
            #         it needs to move to 2 and then 1- so the last will be 1 position)
            #     To wait a bit in every position and if the position is held more than 1 sec than update the score
            
        except:
            pass
        
         #Render some text in the screen
        cv2.putText(image, str(counter), (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.putText(image, position_Right, (10,120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
                
        #On the middle of the screen 
        cv2.putText(image, str(time), (600,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
        #On the right of the screen
        cv2.putText(image, "Frames->" + str(frameCounter), (900,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, position_Left, (900,120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
                    
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()

1280.0
720.0
1280.0
720.0
FPS -> 25.0


error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [9]:
cap.release()
cv2.destroyAllWindows()

In [23]:
cv2.putText??